Import libraries

In [1]:
from bs4 import BeautifulSoup
import os
import datetime
import time
import csv
import sys
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd

Crawl list of companies

In [7]:
DEBUG = 1

def connectchrome():
    options = Options()
    # if you want to hide browser window, uncomment line below
    #options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(executable_path= r'C:\\Users\\user\\Desktop\\python\\banyous\\1-Quora-scrapping\\chromedriver.exe', options = options)
    driver.maximize_window()
    time.sleep(2)
    return driver

driver = connectchrome()
driver.implicitly_wait(3)

# Log-in
driver.get('https://www.businessmodelideas.com/login')
login_button = driver.find_element_by_xpath("//div[@class='col-12 text-lg-right text-left']/button[@type='submit' and @class='cta']")

driver.find_element_by_name('login').send_keys('innolab.admin@ds.seoultech.ac.kr') #user_id
driver.find_element_by_name('password').send_keys('toctoctoc') #password
login_button.click()
driver.implicitly_wait(3)

company_list = []
url1 = "https://www.businessmodelideas.com/explore"

driver.get(url1)
driver.implicitly_wait(3)

# Find company titles (h2 tags) and Save
company_elements = driver.find_elements_by_class_name("company-title")
company_names = [element.text for element in company_elements]

with open('company_list.txt', 'w', encoding='utf-8') as file:
    for name in company_names:
        file.write(name + "\n")

Collecting enterprise-specific data

In [39]:
DEBUG = 1

def connectchrome():
    options = Options()
    # if you want to hide browser window, uncomment line below
    #options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(executable_path= r'C:\\Users\\user\\Desktop\\python\\banyous\\1-Quora-scrapping\\chromedriver.exe', options = options)
    driver.maximize_window()
    time.sleep(2)
    return driver

driver = connectchrome()
driver.implicitly_wait(3)

# Log-in
driver.get('https://www.businessmodelideas.com/login')
login_button = driver.find_element_by_xpath("//div[@class='col-12 text-lg-right text-left']/button[@type='submit' and @class='cta']")

driver.find_element_by_name('login').send_keys('innolab.admin@ds.seoultech.ac.kr') #user_id
driver.find_element_by_name('password').send_keys('toctoctoc') #password
login_button.click()
driver.implicitly_wait(3)

# Read a text file with business names
with open('company_list(N=163).txt', 'r') as file:
    company_names = [line.strip() for line in file.readlines()]
    #company_names = [line.strip().lower() for line in file.readlines()]

# Column headers
headers = [
    "Key Partners", "Key Activities", "Key Resources", "Value Propositions", 
    "Customer Relationships", "Channels", "Customer Segments", 
    "Cost Structure", "Revenue Streams"
]

all_data = []


############################# Pilot
for company_name in company_names:
    company_name = company_name.strip()  # Remove newline characters
    url = f"https://www.businessmodelideas.com/company/{company_name}"
    driver.get(url)

    try:
        # Extract description
        description_element = driver.find_element_by_xpath("//div[@class='description']/p")
        description_text = description_element.text

        data = {"Company": company_name, "Description": description_text}  # Add name and description

        # Collect the rest of the data
        for header in headers:
            span_elements = driver.find_elements_by_xpath(f"//div[@class='cube-heading mb-2' and contains(text(), '{header}')]/following-sibling::div[contains(@class, 'cube-element')]/span")
            span_texts = [element.text for element in span_elements]

            data[header] = span_texts  # Add data for the header

        data['Company'] = company_name  # Add the current business name
        all_data.append(data.copy())  # Add data from the current enterprise to all_data
    except NoSuchElementException:
        print(f"Cannot find data for {company_name}. Skipping this company.")
        continue


# Convert all enterprise data into a single DataFrame
df_all = pd.DataFrame()
for data in all_data:
    df = pd.DataFrame({k: pd.Series(v, dtype='object') for k, v in data.items()})
    df_all = df_all.append(df, ignore_index=True)
    
# DataFrame generation
df = pd.DataFrame(dict([(k, pd.Series(v, dtype='object')) for k,v in data.items()]))

# Save the DataFrame as a CSV file
df_all.to_csv("collected_all_data3.csv", index=False)

# print the number of companies that were successfully collected
print(f"Number of companies successfully scraped: {len(all_data)}")

# Close the Webdriver
driver.quit()

Cannot find data for tupperware. Skipping this company.
Cannot find data for Zalando. Skipping this company.
Cannot find data for forbes. Skipping this company.
Cannot find data for sonos. Skipping this company.
Cannot find data for nike. Skipping this company.
Cannot find data for dollar-shave-club. Skipping this company.
Cannot find data for red-bull. Skipping this company.
Cannot find data for ted. Skipping this company.
Cannot find data for spotify. Skipping this company.
Cannot find data for juul. Skipping this company.
Cannot find data for christian-dior-s-a-. Skipping this company.
Cannot find data for linkedin. Skipping this company.
Cannot find data for mcdonald-s. Skipping this company.
Cannot find data for paypal. Skipping this company.
Cannot find data for facebook. Skipping this company.
Cannot find data for kickstarter. Skipping this company.
Cannot find data for easyjet. Skipping this company.
Cannot find data for solarcity. Skipping this company.
Cannot find data for we

Cannot find data for jet-com. Skipping this company.
Cannot find data for beacon. Skipping this company.
Cannot find data for amiando. Skipping this company.
Cannot find data for myfab. Skipping this company.
Cannot find data for path. Skipping this company.
Cannot find data for kiwi-crate. Skipping this company.
Cannot find data for myvideo. Skipping this company.
Cannot find data for neno-offices. Skipping this company.
Cannot find data for sellaround. Skipping this company.
Number of companies successfully scraped: 7


In [ ]:
data = {"Description": [paragraph_text]}  # Description


for header in headers:
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, f"//div[@class='cube-heading mb-2' and contains(text(), '{header}')]"))
        )
    except TimeoutException:
        print(f"Timeout waiting for header: {header}")
        continue

    # 헤더에 해당하는 cube-element 내부의 span 텍스트들을 추출
    span_elements = driver.find_elements_by_xpath(f"//div[@class='cube-heading mb-2' and contains(text(), '{header}')]/following-sibling::div[contains(@class, 'cube-element')]/span")
    span_texts = [element.text for element in span_elements]

    data[header] = span_texts

# DataFrame generation
df = pd.DataFrame(dict([(k, pd.Series(v, dtype='object')) for k,v in data.items()]))

# Save to .csv file
df.to_csv("collected_data.csv", index=False)

# Close the Webdriver
driver.quit()

In [ ]:
def save_data(data, mode='a'):
    with open("business_ideas.csv", mode, newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        if mode == 'w':
            writer.writerow(["company", "Description", "Employees", "Key partners", "Key Activities", "Key Resources","Value Propositions", "Customer Relationships", "Channels", "Customer Segements", "Cost structure", "Revenue Streams"])  # Header
        writer.writerows(data)
        

In [ ]:
#company_element = driver.title

#if "|" in page_title:
#    company_name = page_title.split("|")[0].strip()
#else:
#    company_name = page_title

## Model Canvas
classes = [
    "boxed-content.cube.big-cube-left", # Key partners
    "boxed-content.cube.small.small-cube-top", # Key Activities
    "boxed-content.cube.cube.small-cube-bottom", # Key Resources
    "boxed-content.cube.big-cube-middle", # Value Propositions
    "boxed-content.cube.cube-small.small-cube-top", # Customer Relationships
    "boxed-content.cube.cube-small.small-cube-bottom", # Channels
    "boxed-content.cube.big-cube-right", # Customer Segments
    "boxed-content.wide-cube-left", # Cost Structure
    "boxed-content.wide-cube-right" # Revenue Streams
]